In [1]:
import h5py
import numpy as np
from scipy.ndimage import label
import neuroglancer

In [2]:
# ------------------ Load HDF5 Volumes ------------------

with h5py.File('../ffn/third_party/karlupia/image_volume.h5', 'r') as f:
    raw = np.squeeze(f['raw'][...])  # Expected dtype: uint8

with h5py.File('../ffn/third_party/karlupia/label_volume.h5', 'r') as f:
    labels = np.squeeze(f['labels'][...])  # Expected dtype: int64

In [4]:
# ------------------ Preprocess for Neuroglancer ------------------

# Neuroglancer expects volumes in X, Y, Z order (we transpose from Z, Y, X)
# raw_ng = np.transpose(raw, (2, 1, 0))
# labels_ng = np.transpose(labels, (2, 1, 0))

# Optional: isolate one segment as a binary seed mask, then relabel
# This mimics what you're doing with `cv.seed > 0` in your example
# Here, we treat nonzero label regions as a single connected component
#seg = (label(labels_ng > 0)[0] == 1).astype(np.uint64)

# ------------------ Neuroglancer Setup ------------------

neuroglancer.set_server_bind_address('0.0.0.0', bind_port=35000)  # to access externally if needed

# Set voxel resolution (update if yours differs!)
dimensions = neuroglancer.CoordinateSpace(
    names=['x', 'y', 'z'],
    units='nm',
    scales=[8, 8, 30],  # Modify to match your voxel spacing
)

viewer = neuroglancer.Viewer()

with viewer.txn() as s:
    s.dimensions = dimensions

    # Add raw image layer (already scaled and uint8 from preprocessing)
    s.layers['raw'] = neuroglancer.ImageLayer(
        source=neuroglancer.LocalVolume(raw, dimensions=dimensions)
    )

    # Add segmentation layer
    s.layers['seg'] = neuroglancer.SegmentationLayer(
        source=neuroglancer.LocalVolume(labels, dimensions=dimensions),
    )

print(f"Neuroglancer is ready! Visit: {viewer}")

Neuroglancer is ready! Visit: http://ip-172-31-64-154.ec2.internal:35000/v/d98d31218ab35caec1747c6fad5faf1c86640481/
